# Evaluating Models

Learning Curves

* It can be shown that *any model* can learn its training data perfectly - “memorize it”.  That is what the blue curve shows below. Any model can achieve a perfect score on the training data as long as it is allowed to be complex enough. Maximum complexity: the model has memorized the entire dataset.

* But memorizing is not the same as learning inherent patterns and use those patterns to make predictions!  Memorization is extremely bad at predicting future outcomes.  See what happens to the red line below as the model starts to memorize the dataset -- the score actually falls!

> Memorization does not generalize well!
    


If we train a model using *training data* and then apply the model to a *validation/test data set* then we obtain these following typical curves:

<!-- ![model curves](assets/model-performance-curves.png) -->

<img src="assets/model-performance-curves.png"  height="200" width="400">

Note: Validation data is data that the model has not seen yet.

Simply put:

1. Undertrained models make a lot of errors on validation data because they have not learned any of the patterns yet.

2. Overtrained models (models that have memorized their training data) make a lot of errors on validation because memorization is extremely bad at predicting the future outcomes.

3. The best models make a trade-off between errors and recognizing important patterns. Notice that for the best models the training score is not 100%!

> In order to find the best model we have to search its *parameter space* in order to find just the right complexity level.

# Train and Test (Validate)

In order to simulate the fact that a model is not able to see all possible data points during training we split our training data into two parts:

* Training data
* Testing (validation) data

We will train our model on the training data as before but we will now test the model performance on the testing data which the model has not seen yet.

> That is, we force the model to make some generalizations.


# Decision Trees: Train and Test

In [1]:
import pandas as pd
from assets.treeviz import tree_print
from sklearn import tree
from sklearn.metrics import accuracy_score
# sklearn provides manipulation of training sets...here we do train/test split
from sklearn.model_selection import train_test_split

# The Iris Data Set

In [2]:
# set up our sklearn data shape for the iris data
df = pd.read_csv("assets/iris.csv")
X  = df.drop(['id','Species'],axis=1)
y = df['Species']

# split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3)

Let's see if we can recreate the results given by the learning curves above.

### Low Complexity Tree

In [3]:
# set up the tree model object - limit the complexity to put us somewhere in the middle of the graph.
model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=1)

# fit the model on the training set of data
model.fit(X_train, y_train)
tree_print(model,X)

# Train results: evaluate the model on the testing set of data
y_train_model = model.predict(X_train)
print("Train Accuracy: {}".format(accuracy_score(y_train, y_train_model)))

# Test results: evaluate the model on the testing set of data
y_test_model = model.predict(X_test)
print("Test Accuracy: {}".format(accuracy_score(y_test, y_test_model)))

if Petal.Width =< 0.75: 
  |then setosa
  |else virginica
<->
Tree Depth:  1
Train Accuracy: 0.6761904761904762
Test Accuracy: 0.6444444444444445


### Medium Complexity Tree

In [4]:
# set up the tree model object - limit the complexity to put us somewhere in the middle of the graph.
model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=4)

# fit the model on the training set of data
model.fit(X_train, y_train)
tree_print(model,X)

# Train results: evaluate the model on the testing set of data
y_train_model = model.predict(X_train)
print("Train Accuracy: {}".format(accuracy_score(y_train, y_train_model)))

# Test results: evaluate the model on the testing set of data
y_test_model = model.predict(X_test)
print("Test Accuracy: {}".format(accuracy_score(y_test, y_test_model)))

if Petal.Length =< 2.450000047683716: 
  |then setosa
  |else if Petal.Length =< 4.75: 
  |  |then if Petal.Width =< 1.6500000953674316: 
  |  |  |then versicolor
  |  |  |else virginica
  |  |else if Petal.Width =< 1.7000000476837158: 
  |  |  |then if Sepal.Width =< 3.049999952316284: 
  |  |  |  |then virginica
  |  |  |  |else versicolor
  |  |  |else virginica
<---------->
Tree Depth:  4
Train Accuracy: 0.9904761904761905
Test Accuracy: 0.9111111111111111


### High Complexity Tree

In [5]:
# set up the tree model object - limit the complexity to put us somewhere in the middle of the graph.
model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=None)

# fit the model on the training set of data
model.fit(X_train, y_train)
tree_print(model,X)

# Train results: evaluate the model on the testing set of data
y_train_model = model.predict(X_train)
print("Train Accuracy: {}".format(accuracy_score(y_train, y_train_model)))

# Test results: evaluate the model on the testing set of data
y_test_model = model.predict(X_test)
print("Test Accuracy: {}".format(accuracy_score(y_test, y_test_model)))

if Petal.Width =< 0.75: 
  |then setosa
  |else if Petal.Length =< 4.75: 
  |  |then if Petal.Width =< 1.6500000953674316: 
  |  |  |then versicolor
  |  |  |else virginica
  |  |else if Petal.Width =< 1.7000000476837158: 
  |  |  |then if Sepal.Width =< 3.049999952316284: 
  |  |  |  |then if Petal.Width =< 1.5499999523162842: 
  |  |  |  |  |then virginica
  |  |  |  |  |else if Petal.Length =< 5.449999809265137: 
  |  |  |  |  |  |then versicolor
  |  |  |  |  |  |else virginica
  |  |  |  |else versicolor
  |  |  |else virginica
<---------------->
Tree Depth:  6
Train Accuracy: 1.0
Test Accuracy: 0.9333333333333333


# Wisconsin Breast Cancer Dataset

The data set is available at <a href="https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)">UCI</a>.
The data set describes benign and malignent tumors based on image measurements.

In [6]:
# set up our sklearn data shape for the iris data
df = pd.read_csv("assets/wdbc.csv")
print(df.shape)
df.head()

(569, 32)


,ID,radius1,texture1,perimeter1,area1,smoothness1,compactness1,concavity1,concave_points1,symmetry1,...,texture3,perimeter3,area3,smoothness3,compactness3,concavity3,concave_points3,symmetry3,fractal_dimension3,Diagnosis
0,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,M
1,2,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,M
2,3,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,M
3,4,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,M
4,5,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,M


In [7]:
# see if our data set is balanced
df['Diagnosis'].value_counts()

B    357
M    212
Name: Diagnosis, dtype: int64

In [8]:
X  = df.drop(['ID','Diagnosis'],axis=1)
y = df['Diagnosis']

# split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3)

### Low Complexity Tree

In [9]:
# set up the tree model object - limit the complexity to put us somewhere in the middle of the graph.
model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=1)

# fit the model on the training set of data
model.fit(X_train, y_train)
tree_print(model,X)

# Train results: evaluate the model on the testing set of data
y_train_model = model.predict(X_train)
print("Train Accuracy: {}".format(accuracy_score(y_train, y_train_model)))

# Test results: evaluate the model on the testing set of data
y_test_model = model.predict(X_test)
print("Test Accuracy: {}".format(accuracy_score(y_test, y_test_model)))

if perimeter3 =< 105.94999694824219: 
  |then B
  |else M
<->
Tree Depth:  1
Train Accuracy: 0.9271356783919598
Test Accuracy: 0.9005847953216374


### Medium Complexity Tree

In [10]:
# set up the tree model object - limit the complexity to put us somewhere in the middle of the graph.
model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=4)

# fit the model on the training set of data
model.fit(X_train, y_train)
tree_print(model,X)

# Train results: evaluate the model on the testing set of data
y_train_model = model.predict(X_train)
print("Train Accuracy: {}".format(accuracy_score(y_train, y_train_model)))

# Test results: evaluate the model on the testing set of data
y_test_model = model.predict(X_test)
print("Test Accuracy: {}".format(accuracy_score(y_test, y_test_model)))

if perimeter3 =< 105.94999694824219: 
  |then if concave_points3 =< 0.11084999889135361: 
  |  |then if area2 =< 48.974998474121094: 
  |  |  |then B
  |  |  |else if radius2 =< 0.6880500316619873: 
  |  |  |  |then M
  |  |  |  |else B
  |  |else if smoothness3 =< 0.1700499951839447: 
  |  |  |then if fractal_dimension3 =< 0.07416000217199326: 
  |  |  |  |then M
  |  |  |  |else B
  |  |  |else M
  |else if concave_points3 =< 0.15074999630451202: 
  |  |then if texture3 =< 19.385000228881836: 
  |  |  |then B
  |  |  |else if radius3 =< 16.799999237060547: 
  |  |  |  |then B
  |  |  |  |else M
  |  |else if concavity1 =< 0.08588500320911407: 
  |  |  |then if radius2 =< 0.2755500078201294: 
  |  |  |  |then B
  |  |  |  |else M
  |  |  |else M
<---------->
Tree Depth:  4
Train Accuracy: 0.9849246231155779
Test Accuracy: 0.9239766081871345


### High Complexity Tree

In [11]:
# set up the tree model object - limit the complexity to put us somewhere in the middle of the graph.
model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=None)

# fit the model on the training set of data
model.fit(X_train, y_train)
tree_print(model,X)

# Train results: evaluate the model on the testing set of data
y_train_model = model.predict(X_train)
print("Train Accuracy: {}".format(accuracy_score(y_train, y_train_model)))

# Test results: evaluate the model on the testing set of data
y_test_model = model.predict(X_test)
print("Test Accuracy: {}".format(accuracy_score(y_test, y_test_model)))

if perimeter3 =< 105.94999694824219: 
  |then if concave_points3 =< 0.11084999889135361: 
  |  |then if area2 =< 48.974998474121094: 
  |  |  |then B
  |  |  |else if radius1 =< 12.265000343322754: 
  |  |  |  |then M
  |  |  |  |else B
  |  |else if smoothness3 =< 0.1700499951839447: 
  |  |  |then if fractal_dimension1 =< 0.05803000181913376: 
  |  |  |  |then M
  |  |  |  |else if symmetry3 =< 0.5001000165939331: 
  |  |  |  |  |then B
  |  |  |  |  |else M
  |  |  |else M
  |else if concave_points3 =< 0.15074999630451202: 
  |  |then if texture3 =< 19.385000228881836: 
  |  |  |then B
  |  |  |else if area3 =< 866.7000122070312: 
  |  |  |  |then if smoothness2 =< 0.005834000185132027: 
  |  |  |  |  |then B
  |  |  |  |  |else M
  |  |  |  |else if concave_points1 =< 0.028039999306201935: 
  |  |  |  |  |then B
  |  |  |  |  |else M
  |  |else if texture3 =< 20.584999084472656: 
  |  |  |then if smoothness3 =< 0.13449999690055847: 
  |  |  |  |then B
  |  |  |  |else M
  |  |  |el

# Train and Test

We have already seen that just using a training set for model evaluation does not work!

Our solution was to split the training data into a training and a test/validation set.

<img src="assets/train-test-data.png" height="200" width="400">

With that we get the typical performance curves that allow us to evaluate models more realistically.


<img src="assets/model-performance-curves.png"  height="200" width="400">

# Problem!

* Train-testing relies on randomly splitting the training data into two parts.

* If this split just happens to be a ‘bad’ split your results might be biased,

**Solution:** cross-validation


# Cross-Validation

* In cross-validation we switch the roles of the two sets
* We evaluate the model on each trial and then take the average
* This will eliminate a lot of the bias

<img src="assets/2fold-xval.png" height="200" width="400">


BUT, what if is the split was really bad: e.g. one of the sets does not contain any examples of one of the classes.

**Solution:** create more trials - *n-fold cross-validation*

As a solution to a single bad split -- perform the split multiple times -- then train and test -- take the average

Example: 
* 5-fold CV - split the training data into 5 splits (folds)
* Use each fold as a test/validation set and the other folds as training set
* Multiple splits - even if one is bad it will be balanced out by the others.

<img src="assets/5fold-xval.png" height="200" width="400">


In [12]:
# cross-validation Iris
import pandas as pd
from sklearn import tree
# grab cross validation code
from sklearn.model_selection import cross_val_score

# get data
df = pd.read_csv("assets/iris.csv")
X  = df.drop(['id','Species'],axis=1)
y = df['Species']

# set up the model
model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=2)

# do the 5-fold cross validation
scores = cross_val_score(model, X, y, cv=5)
print("Fold Accuracies: {}".format(scores))
print("Accuracy: {}".format(scores.mean()))

Fold Accuracies: [ 0.93333333  0.96666667  0.9         0.86666667  1.        ]
Accuracy: 0.9333333333333332


In [13]:
# cross-validation WDBC
import pandas as pd
from sklearn import tree
# grab cross validation code
from sklearn.model_selection import cross_val_score

# get data
df = pd.read_csv("assets/wdbc.csv")
X  = df.drop(['ID','Diagnosis'],axis=1)
y = df['Diagnosis']

# set up the model
model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=3)

# do the 5-fold cross validation
scores = cross_val_score(model, X, y, cv=5)
print("Fold Accuracies: {}".format(scores))
print("Accuracy: {}".format(scores.mean()))

Fold Accuracies: [ 0.90434783  0.91304348  0.94690265  0.9380531   0.95575221]
Accuracy: 0.9316198537899192


# Model Evaluation - the Grid Search

Where did our model parameters come from in the above examples?  We searched for them!  Therefore:

* Finding the best model involves searching for (hyper-)parameter values that give you the best testing/cross-validation accuracy.
* This is usually referred to as the *grid search*.



Sklearn helps us do that efficiently:
* Sklearn has a built-in grid search that can optimize the model parameters
* The tree has two parameters: criterion and depth
* The grid search will find the optimal value for both of these parameters


In [14]:
# Grid search with cross-validation for iris dataset
import pandas as pd
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

# get data
df = pd.read_csv("assets/iris.csv")
X  = df.drop(['id','Species'],axis=1)
y = df['Species']

# setting up grid search
model = model = tree.DecisionTreeClassifier()
param_grid = {'max_depth': list(range(1,11)),
              'criterion': ['entropy', 'gini']
              }
grid = GridSearchCV(model, param_grid, cv=5)

# performing grid search 
grid.fit(X,y)

# print out what we found
print("Best parameters: {}".format(grid.best_params_))

# print out the best model
print("Best tree:")
tree_print(grid.best_estimator_,X)

# Get the accuracy
# Evaluate the tree
# predicting        
predict_y = grid.best_estimator_.predict(X)

# accuracy          
print("Accuracy: {}".format(accuracy_score(y, predict_y)))

Best parameters: {'criterion': 'gini', 'max_depth': 4}
Best tree:
if Petal.Length =< 2.450000047683716: 
  |then setosa
  |else if Petal.Width =< 1.75: 
  |  |then if Petal.Length =< 4.949999809265137: 
  |  |  |then if Petal.Width =< 1.6500000953674316: 
  |  |  |  |then versicolor
  |  |  |  |else virginica
  |  |  |else if Petal.Width =< 1.5499999523162842: 
  |  |  |  |then virginica
  |  |  |  |else versicolor
  |  |else if Petal.Length =< 4.850000381469727: 
  |  |  |then if Sepal.Width =< 3.0999999046325684: 
  |  |  |  |then virginica
  |  |  |  |else versicolor
  |  |  |else virginica
<---------->
Tree Depth:  4
Accuracy: 0.9933333333333333


In [15]:
# Grid search with cross-validation for wisconsin breast cancer dataset
import pandas as pd
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

# get data
df = pd.read_csv("assets/wdbc.csv")
X  = df.drop(['ID','Diagnosis'],axis=1)
y = df['Diagnosis']

# setting up grid search
model = model = tree.DecisionTreeClassifier()
param_grid = {'max_depth': list(range(1,11)),
              'criterion': ['entropy', 'gini']
              }
grid = GridSearchCV(model, param_grid, cv=5)

# performing grid search 
grid.fit(X,y)

# print out what we found
print("Best parameters: {}".format(grid.best_params_))

# print out the best model
print("Best tree:")
tree_print(grid.best_estimator_,X)

# Get the accuracy
# Evaluate the tree
# predicting        
predict_y = grid.best_estimator_.predict(X)

# accuracy          
print("Accuracy: {}".format(accuracy_score(y, predict_y)))

Best parameters: {'criterion': 'entropy', 'max_depth': 4}
Best tree:
if perimeter3 =< 105.94999694824219: 
  |then if concave_points3 =< 0.1350499987602234: 
  |  |then if area2 =< 48.974998474121094: 
  |  |  |then if texture3 =< 30.145000457763672: 
  |  |  |  |then B
  |  |  |  |else B
  |  |  |else if symmetry1 =< 0.16565001010894775: 
  |  |  |  |then M
  |  |  |  |else B
  |  |else if texture3 =< 27.575000762939453: 
  |  |  |then if symmetry3 =< 0.35785001516342163: 
  |  |  |  |then B
  |  |  |  |else M
  |  |  |else M
  |else if perimeter3 =< 117.44999694824219: 
  |  |then if smoothness3 =< 0.13609999418258667: 
  |  |  |then if texture3 =< 25.670000076293945: 
  |  |  |  |then B
  |  |  |  |else M
  |  |  |else if symmetry3 =< 0.2572999894618988: 
  |  |  |  |then B
  |  |  |  |else M
  |  |else if concave_points3 =< 0.08586500585079193: 
  |  |  |then if smoothness3 =< 0.11130999773740768: 
  |  |  |  |then B
  |  |  |  |else M
  |  |  |else M
<---------->
Tree Depth:  4
Ac

# Model Accuracy Reexamined

Consider a classification problem with two classes, then we observe the following outcomes of a prediction of a suitable classification model:

>true positive (TP) -- predicted positive coincides with actual positive

>true negative (TN) -- predicted negative coincides with actual negative

>false positive (FP), Type I error -- predicted positive but actual negative

>false negative (FN), Type II error -- predicted negative but actual positive

Two types of errors possible!


* We can arrange the predictions in a matrix form
* Errors will show up as values outside the major diagonal

<img src="assets/confusion2.png" height="400" width="800">

# The Wisconsin Breast Cancer Data Set

Let's apply everything we have learned so far: build the best model, and then evaluate it.

In [20]:
# set up
import pandas as pd
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix

# get data 
df = pd.read_csv("assets/wdbc.csv")

# create our sklearn data
X  = df.drop(['ID','Diagnosis'],axis=1)
y = df['Diagnosis']

# setting up grid search using 5-fold CV
model = tree.DecisionTreeClassifier()
param_grid = {'max_depth': list(range(1,11)),
              'criterion': ['entropy', 'gini']
              }
grid = GridSearchCV(model, param_grid, cv=5)

# performing grid search 
null = grid.fit(X,y)

# print out what we found
print("Best parameters: {}".format(grid.best_params_))

# print out the best model
print("Best tree:")
tree_print(grid.best_estimator_,X)

# Evaluate the tree
# predicting        
predict_y = grid.best_estimator_.predict(X)

# accuracy          
print("Accuracy: {}".format(accuracy_score(y, predict_y)))

# build the confusion matrix 
labels = ['M','B']
cm = confusion_matrix(y, predict_y, labels=labels)
cm_df = pd.DataFrame(cm, index=labels, columns=labels)
print("Confusion Matrix:\n{}".format(cm_df))

Best parameters: {'criterion': 'entropy', 'max_depth': 4}
Best tree:
if perimeter3 =< 105.94999694824219: 
  |then if concave_points3 =< 0.1350499987602234: 
  |  |then if area2 =< 48.974998474121094: 
  |  |  |then if texture3 =< 30.145000457763672: 
  |  |  |  |then B
  |  |  |  |else B
  |  |  |else if compactness2 =< 0.014419999904930592: 
  |  |  |  |then M
  |  |  |  |else B
  |  |else if texture3 =< 27.575000762939453: 
  |  |  |then if symmetry3 =< 0.35785001516342163: 
  |  |  |  |then B
  |  |  |  |else M
  |  |  |else M
  |else if perimeter3 =< 117.44999694824219: 
  |  |then if smoothness3 =< 0.13609999418258667: 
  |  |  |then if texture3 =< 25.670000076293945: 
  |  |  |  |then B
  |  |  |  |else M
  |  |  |else if perimeter2 =< 1.343999981880188: 
  |  |  |  |then B
  |  |  |  |else M
  |  |else if concave_points3 =< 0.08586500585079193: 
  |  |  |then if texture1 =< 21.259998321533203: 
  |  |  |  |then B
  |  |  |  |else M
  |  |  |else M
<---------->
Tree Depth:  4
Ac

# Midterm 

The midterm is a data analysis project very similar to the projects you have done over the last few weeks.  Find a data set whose **features/independent variables** are all **numerical** and whose **target/dependent variable** is categorical (**only labels - no numbers**, maximum number of different kind of labels is five).  If you have doubts about your selected dataset contact me.  The data set has to be different from all the data sets you have studied so far and you cannot select a dataset that appears in the asset folder of the course notes.  

You have to perform the following tasks on your selected data set:

* preliminary data analysis (use describe and count to analyze basic statistics on your data set).  Briefly report your findings.
* use visualization (especially scatter plots, histograms, and bar graphs) to explore your data set further. Report you findings.
* Build decision tree models. Build the **best** possible decision tree model of your data using the techniques such as grid-search and cross-validation covered in class. 
* Print your best model using treeviz and compute its accuracy
* Compute the confusion matrix and comment on type I and type II errors your model is committing and if you should be concerned about these errors.
* Provide a brief interpretation of the model.  Does it provide any deeper insights or patterns?  Can you relate the  model to patterns you found in your data during visualization?

The midterm is an **INDIVIDUAL** effort, that means each student has to select their own data set and write their own report.  No team work permitted.  Please submit both your notebook and your data set (perhaps as a zip file).